In [3]:
from utils import PkdbModel,measurement_type_data, filter_out, convert_unit
from collections import namedtuple
import pandas as pd
from IPython.display import display, HTML

def multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell    '''
    return HTML(
         '<table><tr style="background-color:white;">' +
         ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
         '</tr></table>'
     )

In [4]:
def midazolam_n_idx(data):
    return (data.substance_intervention == 'midazolam') \
           & (data.substance == 'midazolam') \
           & (data['tissue'] == 'plasma')

In [5]:
def all_midazolam_idx(data, substance="midazolam"):
     return (   (data.substance_intervention == 'midazolam')) \
           & (data["substance"] == substance) \
           & (data['tissue'].isin(['plasma','serum','blood']) )

In [6]:
Inference = namedtuple("Inference",["name","filter_out","measurement_type"])
def inferences(substance):
        return [
    Inference(
        name=f"{substance}_clearance",
        filter_out= {"unit":[ 'microgram * milliliter / hour / micromole']},
        measurement_type = "clearance",
    ),
    Inference(
        name=f"{substance}_timecourse",
        filter_out= {"unit":["mole / liter"]},
        measurement_type = "concentration"
    ),
    Inference(
        name=f"{substance}_thalf",
        filter_out= {},
        measurement_type = "thalf"
    ), 
    Inference(
        name=f"{substance}_tmax",
        filter_out= {},
        measurement_type = "tmax"
    ),
    Inference(
        name=f"{substance}_vd",
        filter_out= {"unit":[ 'liter * milligram / millimole']},
        measurement_type = "vd"
    ),
     Inference(
        name=f"{substance}_auc_inf",
        filter_out= {},
        measurement_type = "auc_inf"
    ),
     Inference(
        name=f"{substance}_auc_end",
        filter_out= {},
        measurement_type = "auc_end"
    ),
    Inference(
        name=f"{substance}_kel",
        filter_out= {},
        measurement_type = "kel"
    ),
]

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
substances = ["midazolam","1-hydroxymidazolam"]


In [9]:
all_complete = PkdbModel("all_complete", destination="2-merged")
all_complete.read()
all_complete.to_numeric()

for substance in substances:
    complete_substance = all_complete.data.loc[all_midazolam_idx(all_complete.data,substance)]
    complete_substance["inferred"] = False
    
    for inference in inferences(substance):
        print(inference.name)
        data = measurement_type_data(complete_substance,inference.measurement_type)
        data_class = PkdbModel(inference.name,destination="3-inferred")
        data_class.data = data
        [data_class.filter_out(k,v)for k,v in inference.filter_out.items()]
        data_class.infer_from_interventions()
        data_class.infer_from_outputs()
        data_class.save()

        t1 = pd.DataFrame(data_class.data.groupby(["unit", "unit_intervention"]).apply(len), columns=["count"])
        t2 = pd.DataFrame(data_class.data.groupby(["inferred"]).apply(len), columns=["count"])
        t3 = pd.DataFrame(data_class.data.groupby(["calculated"]).apply(len), columns=["count"])
        display(multi_table([t1,t2,t3]))

/home/janekg/Env/pkdb_analysis/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


midazolam_clearance


midazolam_timecourse


midazolam_thalf


midazolam_tmax


...

1-hydroxymidazolam_tmax


1-hydroxymidazolam_vd


1-hydroxymidazolam_auc_inf


1-hydroxymidazolam_auc_end


1-hydroxymidazolam_kel
